In [27]:
from lda_model import sent_to_words, process_data_words, build_model,format_topics_sentences,frequency_distribution_word_counts_in_documents


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
filepath='/home/yiyi/Documents/masterthesis/CPD/data/cleand_query_output_csv/019b8823-aa09-4d8f-8b8d-26433d76dff3'
df = pd.read_csv(filepath)

In [18]:
df = df.replace(np.nan, '', regex=True)
df.text = df.text.astype(str)
df.title = df.title.astype(str)
df["words"] = df.title+' '+df.text
en_df = df[df.lang=='en']
en_df= en_df[en_df.words.notnull()]

In [24]:
en_df= en_df[en_df.words.notnull()]

In [25]:
en_df.head()

,uid,date,score,recommendation_rate,lang,title,text,words
0,7041b7b3-f9d2-4d7c-b34b-f9c2bd632671,2009-07-01 00:00:00,60,,en,,Really not worth the money. Nickle-and-dimed f...,Really not worth the money. Nickle-and-dimed ...
1,ecc1ef55-1f8f-44da-bcce-057a29138cae,2009-07-01 00:00:00,100,,en,,Had a fantastic time. Excellent Resort. From...,Had a fantastic time. Excellent Resort. Fro...
2,11ae2de8-7b02-4b17-9a71-b166b77d8365,2010-07-01 00:00:00,100,,en,,We had probably the best stay we've ever had i...,We had probably the best stay we've ever had ...
3,b2230e67-e20e-425d-b8f2-ddf5359bb74b,2009-07-01 00:00:00,80,,en,,"Beautiful Beach, Friendly staff.The Ocean view...","Beautiful Beach, Friendly staff.The Ocean vie..."
4,5e5de5ba-7433-4bcf-b27f-1690255833fd,2010-07-01 00:00:00,100,,en,,"So I only stayed one night but wow, what a nig...","So I only stayed one night but wow, what a ni..."


In [28]:
docs = en_df.words.to_list()  # => data
# process the data words
data_words = list(sent_to_words(docs))

data_ready = process_data_words(data_words)
# build lda model

In [30]:
len(data_ready), len(en_df)

(8069, 8069)

In [34]:
en_df['processed_words']= [" ".join(text) for text in data_ready]

In [35]:
en_df.head(4)

,uid,date,score,recommendation_rate,lang,title,text,words,processed_words
0,7041b7b3-f9d2-4d7c-b34b-f9c2bd632671,2009-07-01 00:00:00,60,,en,,Really not worth the money. Nickle-and-dimed f...,Really not worth the money. Nickle-and-dimed ...,really worth money nickle dim half shower clos...
1,ecc1ef55-1f8f-44da-bcce-057a29138cae,2009-07-01 00:00:00,100,,en,,Had a fantastic time. Excellent Resort. From...,Had a fantastic time. Excellent Resort. Fro...,fantastic time excellent resort check man book...
2,11ae2de8-7b02-4b17-9a71-b166b77d8365,2010-07-01 00:00:00,100,,en,,We had probably the best stay we've ever had i...,We had probably the best stay we've ever had ...,probably best stay resort mid relax enjoy reac...
3,b2230e67-e20e-425d-b8f2-ddf5359bb74b,2009-07-01 00:00:00,80,,en,,"Beautiful Beach, Friendly staff.The Ocean view...","Beautiful Beach, Friendly staff.The Ocean vie...",friendly staff luxury suite smaller expexte co...


In [71]:
rows = []
for text in texts:
    rows.append({"text": text})
df = pd.DataFrame(rows)
print(len(df))

5019


In [36]:
import numpy as np
import tqdm as tqdm

import scipy.sparse as s_sparse

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

from corextopic import corextopic as ct
from corextopic import vis_topic as vt

/home/yiyi/anaconda3/envs/CPD/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/yiyi/anaconda3/envs/CPD/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Topic Modeling

In [37]:
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(en_df['processed_words'])
tfidf = vectorizer.transform(en_df['processed_words'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

5071


In [38]:
from categories import CATEGORIES

In [59]:
categories_lower = [[x.lower() for x in category] for category in CATEGORIES]
len(categories_lower)

15

In [42]:
model = ct.Corex(n_hidden=15, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [43]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: check, call, tell, ask, arrive, hour, wait, pm, finally, put
Topic #2: beach, staff, friendly, great, pool, helpful, clean, private, location, private beach
Topic #3: reach, stay, stay reach, time, resort, first, trip, next, reach resort, towel
Topic #4: bathroom, elevator, floor, shower, dirty, work, door, stain, carpet, furniture
Topic #5: night, find, give, expect, much, really, thing, leave, still, guest
Topic #6: pay, price, bad, property, rate, old, disappointed, average, terrible, poor
Topic #7: restaurant, drink, food, bar, eat, wonderful, dinner, lunch, food drink, set
Topic #8: small, view, large, balcony, area, ocean, water, courtyard, size, pool area
Topic #9: charge, walk, parking, hotel, fee, resort_fee, car, park, include, cost
Topic #10: day, people, bed, morning, start, sit, end, way, entire, day stay
Topic #11: never, year, manager, member, ready, ever, room ready, receive, different, last
Topic #12: review, spend, read, order, week, choose, enough, turn, ex

In [60]:
anchors=[
    [a for a in topic if a in vocab]
    for topic in categories_lower
]
anchors = list(filter(lambda x: len(x)>0, anchors))

In [61]:
anchors.append(["decoration", "renovation", "repair", "restore", "overhaul", "restoration", "renewal",
                "rehabilitation", "revamping", "reimagine", "refurbishment", "doing up", "mordenization", 
                "refitting", "fixing up", "reconditioning"])

In [63]:
len(anchors)

16

In [65]:
model = ct.Corex(n_hidden=16, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [66]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: view, bathroom, balcony, shower, furniture, tv, view room, great view, room view, room size
Topic #2: arrive, give, first, thing, comfort, wait, people, next, guest, bring
Topic #3: ask, cleanliness, pay, expect, bad, point, manager, door, change, experience
Topic #4: beach, private beach, private, beach staff, beach great, hotel beach, beach area, small beach, beach small, seaweed
Topic #5: breakfast, eat breakfast, free breakfast, service breakfast, breakfast lunch, breakfast include, expensive, great breakfast, buffet, restaurant breakfast
Topic #6: food, menu, steak, fish, food drink, great food, food service, seafood, food restaurant, drink food
Topic #7: bar, bar restaurant, pool bar, restaurant bar, drink bar, bar staff, bar pool, bar great, beach bar, bar area
Topic #8: location, parking, great location, location great, shopping, hotel location, location perfect, perfect location, location hotel, perfect
Topic #9: service, room service, management, reception, pool ser

In [68]:

topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(16)]
).astype(float)
topic_df.index = en_df.index
df = pd.concat([en_df, topic_df], axis=1)

In [70]:
anchors

[['bathroom',
  'balcony',
  'view',
  'tv',
  'furniture',
  'family room',
  'shower',
  'room size',
  'old room',
  'new room'],
 ['comfort', 'noise level'],
 ['cleanliness'],
 ['beach'],
 ['breakfast'],
 ['food', 'menu', 'meat', 'steak', 'fish', 'seafood', 'pizza'],
 ['bar'],
 ['location', 'sightseeing', 'shopping', 'parking'],
 ['service',
  'reception',
  'restaurant service',
  'pool service',
  'beach service',
  'housekeeping staff',
  'management',
  'bar service',
  'room service'],
 ['vibe'],
 ['wifi'],
 ['pool'],
 ['fitness'],
 ['value', 'terrace', 'elevator'],
 ['hotel', 'luxury hotel', 'family hotel', 'resort hotel', 'beach hotel'],
 ['decoration',
  'renovation',
  'repair',
  'restore',
  'overhaul',
  'restoration',
  'renewal',
  'rehabilitation',
  'revamping',
  'reimagine',
  'refurbishment',
  'doing up',
  'mordenization',
  'refitting',
  'fixing up',
  'reconditioning']]

In [73]:
df = df.sort_values('date')

In [74]:
# df.to_csv('test.csv')